In [82]:
import pandas as pd

In [83]:
# application_test = pd.read_csv(r"..\data\raw\application_train.csv")
# installments_payments = pd.read_csv(r"..\data\raw\installments_payments.csv")
# credit_card_balance = pd.read_csv(r"..\data\raw\credit_card_balance.csv")
# bureau = pd.read_csv(r"..\data\raw\bureau.csv")
# bureau_balance = pd.read_csv(r"..\data\raw\bureau_balance.csv")



In [84]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# 1. PROCESS BUREAU AND BUREAU_BALANCE DATA
def process_bureau_and_balance(df):
    """Processes bureau.csv and bureau_balance.csv to create aggregated features."""
    bureau = pd.read_csv('../data/raw/bureau.csv')
    bureau_balance = pd.read_csv('../data/raw/bureau_balance.csv')

    # Aggregate bureau_balance by SK_ID_BUREAU
    bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg({
        'MONTHS_BALANCE': ['min', 'max', 'count'],
        'STATUS': ['first', 'last']
    }).reset_index()
    bb_agg.columns = ['_'.join(col).strip('_') for col in bb_agg.columns.values]
    
    # Merge aggregated bureau_balance into bureau
    bureau = bureau.merge(bb_agg, how='left', on='SK_ID_BUREAU')
    
    # Aggregate bureau by SK_ID_CURR
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({
        'SK_ID_BUREAU': 'count',
        'CREDIT_DAY_OVERDUE': ['mean', 'max'],
        'AMT_CREDIT_SUM_DEBT': 'sum',
        'AMT_CREDIT_SUM_OVERDUE': 'sum',
        'CREDIT_TYPE': 'nunique'
    }).reset_index()
    bureau_agg.columns = ['SK_ID_CURR'] + [f'BUREAU_{col[0]}_{col[1].upper()}' for col in bureau_agg.columns[1:]]
    
    return df.merge(bureau_agg, on='SK_ID_CURR', how='left')

# 2. PROCESS PREVIOUS APPLICATIONS DATA
def process_previous_applications(df):
    """Processes previous_application.csv for historical application features."""
    prev_app = pd.read_csv('../data/raw/previous_application.csv')
    
    # Feature engineering for previous applications
    prev_app['APP_CREDIT_PERC'] = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT']
    
    # Aggregate by SK_ID_CURR
    prev_app_agg = prev_app.groupby('SK_ID_CURR').agg({
        'SK_ID_PREV': 'count',
        'AMT_ANNUITY': ['mean', 'max'],
        'AMT_APPLICATION': ['mean', 'max'],
        'APP_CREDIT_PERC': ['mean', 'max', 'min']
    }).reset_index()
    prev_app_agg.columns = ['SK_ID_CURR'] + [f'PREV_APP_{col[0]}_{col[1].upper()}' for col in prev_app_agg.columns[1:]]

    return df.merge(prev_app_agg, on='SK_ID_CURR', how='left')

# 3. PROCESS INSTALLMENTS PAYMENTS DATA (CRUCIAL FOR DPD)
def process_installments_payments(df):
    """Processes installments_payments.csv to get DPD and payment history."""
    installments = pd.read_csv('../data/raw/installments_payments.csv')

    # Calculate DPD and payment difference
    installments['DPD'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']
    installments['DBD'] = installments['DAYS_INSTALMENT'] - installments['DAYS_ENTRY_PAYMENT']
    installments['PAYMENT_PERC'] = installments['AMT_PAYMENT'] / installments['AMT_INSTALMENT']
    installments['PAYMENT_DIFF'] = installments['AMT_INSTALMENT'] - installments['AMT_PAYMENT']
    
    # Flag late payments
    installments['DPD'] = installments['DPD'].apply(lambda x: x if x > 0 else 0)
    installments['DBD'] = installments['DBD'].apply(lambda x: x if x > 0 else 0)

    # Aggregate by SK_ID_CURR
    installments_agg = installments.groupby('SK_ID_CURR').agg({
        'DPD': ['mean', 'max', 'sum'],
        'DBD': ['mean', 'max', 'sum'],
        'PAYMENT_PERC': ['mean', 'max', 'min'],
        'PAYMENT_DIFF': ['mean', 'max', 'sum']
    }).reset_index()
    installments_agg.columns = ['SK_ID_CURR'] + [f'INSTALLMENTS_{col[0]}_{col[1].upper()}' for col in installments_agg.columns[1:]]

    return df.merge(installments_agg, on='SK_ID_CURR', how='left')

# MAIN FUNCTION TO BUILD THE DATASET
def build_complete_dataset():
    """Loads base data and merges all engineered features."""
    df_train = pd.read_csv('../data/raw/application_train.csv')
    df_test = pd.read_csv('../data/raw/application_test.csv')
    df = pd.concat([df_train, df_test], ignore_index=True)
    df = df[['SK_ID_CURR','TARGET','NAME_CONTRACT_TYPE','AMT_INCOME_TOTAL','AMT_CREDIT','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]

    print("Processing Bureau Data...")
    df = process_bureau_and_balance(df)
    
    print("Processing Previous Applications...")
    df = process_previous_applications(df)
    
    print("Processing Installments Payments...")
    df = process_installments_payments(df)
    
    # You can add calls to process credit_card_balance and POS_CASH_balance here as well
    
    print("Dataset build complete.")
    return df

# Run the pipeline
complete_df = build_complete_dataset()
print("Shape of the final dataset:", complete_df.shape)
print("Columns created:", [col for col in complete_df.columns if 'BUREAU_' in col or 'PREV_APP_' in col or 'INSTALLMENTS_' in col])

Processing Bureau Data...
Processing Previous Applications...
Processing Previous Applications...
Processing Installments Payments...
Processing Installments Payments...
Dataset build complete.
Shape of the final dataset: (356255, 39)
Columns created: ['BUREAU_SK_ID_BUREAU_COUNT', 'BUREAU_CREDIT_DAY_OVERDUE_MEAN', 'BUREAU_CREDIT_DAY_OVERDUE_MAX', 'BUREAU_AMT_CREDIT_SUM_DEBT_SUM', 'BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM', 'BUREAU_CREDIT_TYPE_NUNIQUE', 'PREV_APP_SK_ID_PREV_COUNT', 'PREV_APP_AMT_ANNUITY_MEAN', 'PREV_APP_AMT_ANNUITY_MAX', 'PREV_APP_AMT_APPLICATION_MEAN', 'PREV_APP_AMT_APPLICATION_MAX', 'PREV_APP_APP_CREDIT_PERC_MEAN', 'PREV_APP_APP_CREDIT_PERC_MAX', 'PREV_APP_APP_CREDIT_PERC_MIN', 'INSTALLMENTS_DPD_MEAN', 'INSTALLMENTS_DPD_MAX', 'INSTALLMENTS_DPD_SUM', 'INSTALLMENTS_DBD_MEAN', 'INSTALLMENTS_DBD_MAX', 'INSTALLMENTS_DBD_SUM', 'INSTALLMENTS_PAYMENT_PERC_MEAN', 'INSTALLMENTS_PAYMENT_PERC_MAX', 'INSTALLMENTS_PAYMENT_PERC_MIN', 'INSTALLMENTS_PAYMENT_DIFF_MEAN', 'INSTALLMENTS_PAYMENT_

In [85]:
import numpy as np
import pandas as pd

# Assume 'complete_df' is your fully built dataset from the previous steps
# For this example, let's create a placeholder if you don't have it loaded.
# complete_df = pd.read_csv('path_to_your_complete_df.csv') 

# Fill missing DPD values with 0 for the logical comparison
complete_df['INSTALLMENTS_DPD_MAX'] = complete_df['INSTALLMENTS_DPD_MAX'].fillna(0)
complete_df['BUREAU_CREDIT_DAY_OVERDUE_MAX'] = complete_df['BUREAU_CREDIT_DAY_OVERDUE_MAX'].fillna(0)

# Define the conditions for each stage
conditions = [
    # Condition for Stage 3: The loan has defaulted.
    complete_df['TARGET'] == 1,
    
    # Condition for Stage 2: Not defaulted BUT has high DPD either historically or currently.
    (complete_df['TARGET'] == 0) & 
    ((complete_df['INSTALLMENTS_DPD_MAX'] > 30) | (complete_df['BUREAU_CREDIT_DAY_OVERDUE_MAX'] > 30)),
]

# Define the corresponding stage values
choices = [3, 2]

# Create the 'STAGE' column using np.select
# The default value is 1 for all loans that don't meet the other conditions.
complete_df['STAGE'] = np.select(conditions, choices, default=1)

# Display the distribution of the newly created stages
print("Distribution of IFRS 9 Stages:")

print(complete_df['STAGE'].value_counts())

Distribution of IFRS 9 Stages:
STAGE
1    313052
3     24825
2     18378
Name: count, dtype: int64


Cleaning

In [86]:
# Create c_EXT_SOURCE using coalesce logic
complete_df['c_EXT_SOURCE'] = complete_df['EXT_SOURCE_1'].fillna(
    complete_df['EXT_SOURCE_2'].fillna(
        complete_df['EXT_SOURCE_3']
    )
)

# Reorder columns to place c_EXT_SOURCE before EXT_SOURCE_1
cols = list(complete_df.columns)
ext_source_1_idx = cols.index('EXT_SOURCE_1')
c_ext_source_idx = cols.index('c_EXT_SOURCE')
cols.pop(c_ext_source_idx)
cols.insert(ext_source_1_idx, 'c_EXT_SOURCE')
complete_df = complete_df[cols]

Issue Monitoring

In [87]:
# issue1: if no Target then remove
def issue_rate(issue):
    rate = round(len(issue)/len(complete_df),2)
    return print(f"Issue rate: {rate}% ({len(issue)}/{len(complete_df)})")

a = complete_df['TARGET'].isna()
issue1 = complete_df[a]
issue_rate(issue1)
complete_df = complete_df[~a]

# if all three EXT_SOURCE are NaN then remove
a = complete_df['c_EXT_SOURCE'].isna()
issue2 = complete_df[a]
issue_rate(issue2)
complete_df = complete_df[~a]

# if BUREAU_CREDIT_DAY_OVERDUE_MEAN is NaN then remove
a = complete_df['BUREAU_CREDIT_DAY_OVERDUE_MEAN'].isna()
issue3 = complete_df[a]
issue_rate(issue3)
complete_df = complete_df[~a]

# if INSTALLMENTS_DPD_MEAN is NaN then remove
a = complete_df['INSTALLMENTS_DPD_MEAN'].isna()
issue4 = complete_df[a]
issue_rate(issue4)
complete_df = complete_df[~a]

Issue rate: 0.14% (48744/356255)
Issue rate: 0.0% (172/307511)
Issue rate: 0.0% (172/307511)
Issue rate: 0.14% (43932/307339)
Issue rate: 0.14% (43932/307339)
Issue rate: 0.05% (13436/263407)
Issue rate: 0.05% (13436/263407)


TEMP

In [76]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Data columns (total 41 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   SK_ID_CURR                         356255 non-null  int64  
 1   TARGET                             307511 non-null  float64
 2   NAME_CONTRACT_TYPE                 356255 non-null  object 
 3   AMT_INCOME_TOTAL                   356255 non-null  float64
 4   AMT_CREDIT                         356255 non-null  float64
 5   NAME_INCOME_TYPE                   356255 non-null  object 
 6   NAME_EDUCATION_TYPE                356255 non-null  object 
 7   NAME_FAMILY_STATUS                 356255 non-null  object 
 8   NAME_HOUSING_TYPE                  356255 non-null  object 
 9   OCCUPATION_TYPE                    244259 non-null  object 
 10  c_EXT_SOURCE                       356076 non-null  float64
 11  EXT_SOURCE_1                       1623

In [77]:
complete_df.isnull().sum().reset_index()

,index,0
0,SK_ID_CURR,0
1,TARGET,48744
2,NAME_CONTRACT_TYPE,0
3,AMT_INCOME_TOTAL,0
4,AMT_CREDIT,0
5,NAME_INCOME_TYPE,0
6,NAME_EDUCATION_TYPE,0
7,NAME_FAMILY_STATUS,0
8,NAME_HOUSING_TYPE,0
9,OCCUPATION_TYPE,111996


In [81]:
complete_df[complete_df['INSTALLMENTS_DPD_MEAN'].isna()]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,INSTALLMENTS_DBD_MEAN,INSTALLMENTS_DBD_MAX,INSTALLMENTS_DBD_SUM,INSTALLMENTS_PAYMENT_PERC_MEAN,INSTALLMENTS_PAYMENT_PERC_MAX,INSTALLMENTS_PAYMENT_PERC_MIN,INSTALLMENTS_PAYMENT_DIFF_MEAN,INSTALLMENTS_PAYMENT_DIFF_MAX,INSTALLMENTS_PAYMENT_DIFF_SUM,STAGE
26,100031,1.0,Cash loans,112500.0,979992.0,Working,Secondary / secondary special,Widow,House / apartment,Cooking staff,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
50,100058,0.0,Revolving loans,54000.0,135000.0,State servant,Higher education,Married,House / apartment,Core staff,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
64,100076,0.0,Cash loans,180000.0,315000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,Drivers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
81,100096,1.0,Cash loans,81000.0,252000.0,Pensioner,Secondary / secondary special,Married,House / apartment,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
87,100102,0.0,Cash loans,126000.0,327024.0,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307117,455813,0.0,Cash loans,108000.0,668304.0,Working,Secondary / secondary special,Married,House / apartment,High skill tech staff,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
307422,456155,0.0,Cash loans,202500.0,1971072.0,Pensioner,Higher education,Married,House / apartment,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
307439,456177,0.0,Cash loans,247500.0,900000.0,Working,Secondary / secondary special,Married,House / apartment,Managers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
307449,456187,0.0,Cash loans,315000.0,1175314.5,Working,Incomplete higher,Married,House / apartment,Sales staff,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
